In [4]:
import requests
import threading
import feedparser
import pandas as pd
import time

# Time limit in seconds for each thread
TIME_LIMIT = 60

def fetch_articles(source, url):
    print(f"Fetching articles from: {url}")

    article_data = []
    max_articles = 500
    start_time = time.time()  # Record the start time

    try:
        while len(article_data) < max_articles:
            # Check if the elapsed time exceeds the TIME_LIMIT
            elapsed_time = time.time() - start_time
            if elapsed_time > TIME_LIMIT:
                print(f"Time limit reached for {source}. Stopping early.")
                break

            response = requests.get(url)
            response.raise_for_status()
            feed = feedparser.parse(response.content)

            articles = feed.entries

            if not articles:
                print(f"No more articles available from {source}.")
                break

            # Append articles until the limit is reached
            for article in articles:
                if len(article_data) >= max_articles:  # Stop if we have enough articles
                    print(f"Reached 500 articles for {source}.")
                    break
                article_info = {
                    'title': article.title,
                    'link': article.link,
                    'published': getattr(article, 'published', 'No date available'),
                    'description': article.get('description', 'No description available')
                }
                article_data.append(article_info)

            print(f"Fetched {len(article_data)} articles from {source}. Total collected: {len(article_data)}")
            time.sleep(1)  # Sleep to avoid overwhelming the server

    except Exception as e:
        print(f"Error fetching articles from {url}: {e}")

    # Save to CSV regardless of whether it's interrupted or completed
    df = pd.DataFrame(article_data)
    df.to_csv(f'{source}_articles.csv', index=False)
    print(f"Saved articles from {source} to {source}_articles.csv")

def controlScraping(websites):
    threads = []
    try:
        for website, url in websites.items():
            t = threading.Thread(target=fetch_articles, args=(website, url))
            t.start()
            threads.append(t)

        for t in threads:
            t.join()  # Wait for all threads to complete or exit early

    except KeyboardInterrupt:
        print("KeyboardInterrupt received. Stopping and saving current progress...")
        for t in threads:
            t.join()  # Ensure all threads stop

def main():
    websites = {
        'BBC': 'http://feeds.bbci.co.uk/news/rss.xml',
        'CNN': 'http://rss.cnn.com/rss/edition.rss',
        'The Guardian': 'https://www.theguardian.com/uk/rss',
        'New York Times': 'https://rss.nytimes.com/services/xml/rss/nyt/HomePage.xml',
        'Al Jazeera': 'https://www.aljazeera.com/xml/rss/all.xml'
    }
    controlScraping(websites)

if __name__ == '__main__':
    main()



Fetching articles from: http://feeds.bbci.co.uk/news/rss.xml
Fetching articles from: http://rss.cnn.com/rss/edition.rss
Fetching articles from: https://www.theguardian.com/uk/rss
Fetching articles from: https://rss.nytimes.com/services/xml/rss/nyt/HomePage.xml
Fetching articles from: https://www.aljazeera.com/xml/rss/all.xml
Fetched 25 articles from Al Jazeera. Total collected: 25
Fetched 33 articles from BBC. Total collected: 33Fetched 25 articles from New York Times. Total collected: 25

Fetched 50 articles from CNN. Total collected: 50
Fetched 117 articles from The Guardian. Total collected: 117
Fetched 50 articles from Al Jazeera. Total collected: 50
Fetched 50 articles from New York Times. Total collected: 50
Fetched 66 articles from BBC. Total collected: 66
Fetched 100 articles from CNN. Total collected: 100
Fetched 234 articles from The Guardian. Total collected: 234
Fetched 75 articles from Al Jazeera. Total collected: 75
Fetched 75 articles from New York Times. Total collected